# GEC Baseline - Transformer

## Downloading required packages

In [ ]:
!pip install datasets
!pip install wandb

In [ ]:
!pip install -U accelerate
!pip install -U transformers

## Importing required packages

In [ ]:
import os

import torch

import wandb

import transformers
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoConfig
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

import datasets
from datasets import load_dataset, load_metric

## WandB Login

In [ ]:
wandb.login()

## Data Preprocessing


### Dataset structure
The fields we care about in the dataset are structured as below
- **text**: gramatically incorrect text (input to model)
- **edits**: Each edit is a span `[start:end]` where the original text should be replaced by `edits.text`
  - **start**: start indexes of each edit as a list of integers
  - **end**: end indexes of each edit as a list of integers
  - **text**: the text content of each edit as a list of strings

### Tokenize + Create labels

The inputs and labels for the transformer are as below

- **Inputs** = grammatically incorrect text
- **Labels** = corrected text

1. I tokenize and divide each input text into smaller chunks of length at most `max_len`. A `stride` is used to create overlaps between windows (so that the model learns grammar errors that occur across boundaries of chunks). These are now the inputs to the model.
2. Then, I compute the appropriate corrected text for each "chunk" and tokenize these. These become the labels for the model.

- At the moment, I'm replacing corrections that are `None` with `tokenizer.unk_token`. An example where this occurs is printed at the end

In [ ]:
raw_datasets = load_dataset("wi_locness", "wi")

In [ ]:
model_checkpoint = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def preprocess_function(examples):
    inputs = examples['text']
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        return_offsets_mapping=True
    )
    
    labels_out = []
    offset_mapping = model_inputs.pop("offset_mapping")
    for i in range(len(model_inputs["input_ids"])):
        example_idx = i

        start_idx = offset_mapping[i][0][0]
        end_idx = offset_mapping[i][-2][1]  # last token is <eos>, so we care about second last tok offset

        edits = examples["edits"][example_idx]

        corrected_text = inputs[example_idx][start_idx:end_idx]

        for start, end, correction in reversed(
            list(zip(edits["start"], edits["end"], edits["text"]))
        ):
            if start < start_idx or end > end_idx:
                continue
            start_offset = start - start_idx  # >= 0
            end_offset = end - start_idx
            if correction == None:
                correction = tokenizer.unk_token
            corrected_text = (
                corrected_text[:start_offset] + correction + corrected_text[end_offset:]
            )
        
        labels_out.append(corrected_text)
    
    labels_out = tokenizer(labels_out, max_length=512, truncation=True)
    model_inputs["labels"] = labels_out["input_ids"]
    
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets['train'].column_names
)

In [ ]:
# Train-Test split of 90%-10%
dataset_dict = tokenized_datasets["train"].train_test_split(test_size=0.1, seed=0)
tokenized_datasets["train"] = dataset_dict["train"]
tokenized_datasets["test"] = dataset_dict["test"]

### Batching
`DataCollatorForSeq2Seq` will automatically pad the input texts to the same size and batch them for training

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

## Model

### Creating the model



In [ ]:
# Initialize model

config = AutoConfig.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_config(config)

### Set WandB params

In [ ]:
# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"] = "end"
run = wandb.init(entity="ay2324s2-cs4248-team-47", project="gec-baseline-transformer")

### Set training arguments

In [ ]:
output_dir = 'baseline-transformer'

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	auto_find_batch_size=True,
    num_train_epochs=100,
    fp16=True,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="steps",
    eval_steps = 0.1,
    save_strategy="steps",
    save_steps = 0.1,
    load_best_model_at_end=True,
    report_to="wandb"
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"]
)

model.config.use_cache = False

### Train the model

In [ ]:
# !jupyter notebook --ServerApp.iopub_data_rate_limit=1.0e10

In [ ]:
trainer.train()
trainer.save_model(output_dir)

In [ ]:
wandb.finish()